In [1]:
###now try to use keras to train the model
###try deep learning 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import PredefinedSplit
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.svm import SVC
import tensorflow as tf
import keras
import pickle
import sklearn
import sys
import cleanup
import datetime
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import gensim 
from sklearn.multiclass import OneVsRestClassifier
####use a pretrained wordembedding
print("sklearn version:",sklearn.__version__)
print("keras version:",keras.__version__)
print("tensorflow version:",tf.__version__)
print("pandas version:",pd.__version__)
print("numpy version:",np.__version__)
print("python version:",sys.version)
clean=cleanup.cleanup(stem_on=True)

Using TensorFlow backend.


sklearn version: 0.20.2
keras version: 2.2.4
tensorflow version: 1.12.0
pandas version: 0.23.4
numpy version: 1.15.4
python version: 3.6.7 |Anaconda custom (64-bit)| (default, Oct 23 2018, 14:01:38) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
df=pd.read_csv("complete_math_arxiv.csv")

/Users/ben/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### We need to preprocess the dataframe

In [3]:
df.head()

,Unnamed: 0,abstract,authors,categories,created,id,title,updated,clean_abstract,math.MP,...,math.AC,math.LO,math.MG,math.SG,math.SP,math.CT,math.KT,math.GN,math.GM,math.HO
0,0,We construct a generic extension in which the ...,"['Thomas Jech', 'Saharon Shelah']",['math.LO'],1989-04-14,math/9201239,A note on canonical functions,1989-04-14,construct gener extens aleph canon function al...,0,...,0,1,0,0,0,0,0,0,0,0
1,1,It is shown that if $C$ is an $n$-dimensional ...,['Keith Ball'],"['math.MG', 'math.FA']",1989-10-26,math/9201205,Volume ratios and a reverse isoperimetric ineq...,1989-10-26,shown dimension convex bodi affin imag larger ...,0,...,0,0,1,0,0,0,0,0,0,0
2,2,It is proved that if $C$ is a convex body in $...,['Keith Ball'],"['math.MG', 'math.FA']",1989-10-26,math/9201204,Shadows of convex bodies,1989-10-26,prove convex bodi affin imag zero volum codime...,0,...,0,0,1,0,0,0,0,0,0,0
3,3,"It is proved that if $u_1,\ldots, u_n$ are vec...","['Keith Ball', 'Alain Pajor']","['math.MG', 'math.FA']",1989-10-26,math/9201203,Convex bodies with few faces,1989-10-26,prove vector volum symmetr convex bodi whose b...,0,...,0,0,1,0,0,0,0,0,0,0
4,4,"This note deals with the following problem, th...","['Gideon Schechtman', 'Joel Zinn']","['math.FA', 'math.MG']",1989-11-09,math/9201206,On the volume of the intersection of two $L_p^...,1989-11-09,note deal follow problem case introduc vitali ...,0,...,0,0,1,0,0,0,0,0,0,0


In [4]:
#####Preprocess
def list_of_authors(text):
    names=[]
    name_list=text.split(',')
    for name in name_list:
        name=name.strip()
        if name[0]=='[':
            name=name[1:]
        if name[-1]==']':
            name=name[:-1]
        name=name[1:-1]
        names+=[name]
    return names    


def list_of_categories(text):
    pat=re.compile(r'math.[A-Z][A-Z]')
    return pat.findall(text)

df['categories']=df['categories'].apply(lambda x: list_of_categories(x))
df['authors']=df['authors'].apply(lambda x: list_of_authors(x))
df['created']=df['created'].apply(lambda x:datetime.datetime.strptime(x, "%Y-%m-%d"))
if "clean_abstract" not in df.columns:
    df['clean_abstract']=df['abstract'].apply(lambda x: clean.transform(x))
      
all_cat={}
for i in df.index:
    paper_cats=df.loc[i,"categories"]
    for cat in paper_cats:
        if cat in all_cat:
            all_cat[cat]+=1
        else:
            all_cat[cat]=1

list_of_all_cat=sorted(list(all_cat.items()),key=lambda x: x[1], reverse=True)            
all_cats=list(zip(*list_of_all_cat))[0]
index_to_cat={}
for i in range(len(all_cats)):
    index_to_cat[i]=all_cats[i]    

for i in range(len(index_to_cat)):
    df[index_to_cat[i]]=df["categories"].apply(lambda x: 1 if index_to_cat[i] in x else 0)

      
###make 

df=df[df['clean_abstract'].apply(lambda x : type(x)!=type(3.0))]
df=df[df['categories'].apply(lambda x : len(x)>0)]    
df=df.reset_index().drop(columns=['index'])

In [5]:
df.head()

,Unnamed: 0,abstract,authors,categories,created,id,title,updated,clean_abstract,math.MP,...,math.AC,math.LO,math.MG,math.SG,math.SP,math.CT,math.KT,math.GN,math.GM,math.HO
0,0,We construct a generic extension in which the ...,"[Thomas Jech, Saharon Shelah]",[math.LO],1989-04-14,math/9201239,A note on canonical functions,1989-04-14,construct gener extens aleph canon function al...,0,...,0,1,0,0,0,0,0,0,0,0
1,1,It is shown that if $C$ is an $n$-dimensional ...,[Keith Ball],"[math.MG, math.FA]",1989-10-26,math/9201205,Volume ratios and a reverse isoperimetric ineq...,1989-10-26,shown dimension convex bodi affin imag larger ...,0,...,0,0,1,0,0,0,0,0,0,0
2,2,It is proved that if $C$ is a convex body in $...,[Keith Ball],"[math.MG, math.FA]",1989-10-26,math/9201204,Shadows of convex bodies,1989-10-26,prove convex bodi affin imag zero volum codime...,0,...,0,0,1,0,0,0,0,0,0,0
3,3,"It is proved that if $u_1,\ldots, u_n$ are vec...","[Keith Ball, Alain Pajor]","[math.MG, math.FA]",1989-10-26,math/9201203,Convex bodies with few faces,1989-10-26,prove vector volum symmetr convex bodi whose b...,0,...,0,0,1,0,0,0,0,0,0,0
4,4,"This note deals with the following problem, th...","[Gideon Schechtman, Joel Zinn]","[math.FA, math.MG]",1989-11-09,math/9201206,On the volume of the intersection of two $L_p^...,1989-11-09,note deal follow problem case introduc vitali ...,0,...,0,0,1,0,0,0,0,0,0,0


In [8]:
train=df[df['created']<datetime.datetime(year=2018,month=1,day=1)]
test=df[df['created']>=datetime.datetime(year=2018,month=1,day=1)]
num_class=len(index_to_cat)

In [9]:
y_train = train.apply(lambda row:[row[index_to_cat[i]] for i in range(num_class)],axis=1)
y_train = np.array(list(y_train))
y_test = test.apply(lambda row:[row[index_to_cat[i]] for i in range(num_class)],axis=1)
y_test = np.array(list(y_test))

In [11]:
tdift = TfidfVectorizer(stop_words='english',ngram_range=(1,3),min_df=2,max_df=0.7)
tdift.fit(train['clean_abstract'])
X_train = tdift.transform(train['clean_abstract'])
X_test = tdift.transform(test['clean_abstract'])

In [13]:
###build two layers of nn to train 

In [14]:
y_train.shape

(351473, 32)

In [15]:
X_train.shape

(351473, 2984160)

In [19]:
lr_multi=OneVsRestClassifier(LogisticRegression(),n_jobs=-1)

In [20]:
lr_multi.fit(X_train,y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=-1)

In [21]:
###this is the subset accuracy which is a harsh metric since you require for each sample that each label set be correctly predicted.
print(lr_multi.score(X_test,y_test))

0.5029996147283835


In [22]:
pickle.dump(tdift, open("tdift.pkl", 'wb'))

In [23]:
pickle.dump(lr_multi, open("after_tdift_lr_multi.pkl", 'wb'))

In [24]:
tdift = pickle.load(open("tdift.pkl", 'rb'))
lr_multi = pickle.load(open("after_tdift_lr_multi.pkl", 'rb'))
